In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_before = pd.read_excel('total.xlsx', sheet_name="before")
df_during = pd.read_excel('total.xlsx', sheet_name="during")
df_after = pd.read_excel('total.xlsx', sheet_name="after")

In [ ]:
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests

# Chia dataframe thành các phần tương ứng với các giai đoạn
before_covid = df_before
during_covid = df_during
after_period = df_after

# Danh sách các biến liên quan đến abnormal_return (AR)
independent_vars = ['herding_behavior', 'loss_aversion', 'mental_accounting', 'overconfidence']

# Hàm kiểm định Granger cho từng cặp biến và giai đoạn
def granger_test(data, var1, var2, maxlag=4):
    result = grangercausalitytests(data[[var1, var2]], maxlag=maxlag, verbose=False)
    f_stat = result[1][0]['ssr_ftest'][0]
    prob = result[1][0]['ssr_ftest'][1]
    return f_stat, prob

# Khởi tạo kết quả
results = []

# Kiểm định cho tất cả các cặp biến mà AR là biến phụ thuộc
for var in independent_vars:
    # AR does not Granger Cause the independent variable
    f_stat_ar_to_indep, prob_ar_to_indep = granger_test(before_covid, 'abnormal_return', var)
    # Independent variable does not Granger Cause AR
    f_stat_indep_to_ar, prob_indep_to_ar = granger_test(before_covid, var, 'abnormal_return')

    # During COVID-19
    f_stat_ar_to_indep_d, prob_ar_to_indep_d = granger_test(during_covid, 'abnormal_return', var)
    f_stat_indep_to_ar_d, prob_indep_to_ar_d = granger_test(during_covid, var, 'abnormal_return')

    # Full period
    f_stat_ar_to_indep_f, prob_ar_to_indep_f = granger_test(after_period, 'abnormal_return', var)
    f_stat_indep_to_ar_f, prob_indep_to_ar_f = granger_test(after_period, var, 'abnormal_return')

    # Thêm kết quả vào danh sách
    results.append({
        'Null hypothesis': f'AR does not Granger Cause {var}',
        'Before COVID-19 F-stat': f_stat_ar_to_indep,
        'Before COVID-19 Prob': prob_ar_to_indep,
        'During COVID-19 F-stat': f_stat_ar_to_indep_d,
        'During COVID-19 Prob': prob_ar_to_indep_d,
        'After period F-stat': f_stat_ar_to_indep_f,
        'After period Prob': prob_ar_to_indep_f
    })
    results.append({
        'Null hypothesis': f'{var} does not Granger Cause AR',
        'Before COVID-19 F-stat': f_stat_indep_to_ar,
        'Before COVID-19 Prob': prob_indep_to_ar,
        'During COVID-19 F-stat': f_stat_indep_to_ar_d,
        'During COVID-19 Prob': prob_indep_to_ar_d,
        'After period F-stat': f_stat_indep_to_ar_f,
        'After period Prob': prob_indep_to_ar_f
    })

# Chuyển kết quả thành DataFrame để dễ dàng trình bày
result_df = pd.DataFrame(results)

# In kết quả
print(result_df)


In [ ]:
print(result_df)

In [ ]:
result_df = pd.DataFrame(results)

In [ ]:
result_df.to_excel(r'C:\Users\DELL\Desktop\financial materials\Granger_test_results.xlsx')

In [ ]:
result_df

## Linear Regression OLS

In [ ]:
!pip install statsmodels --upgrade

In [21]:
import pandas as pd
import statsmodels.api as sm

# Đọc dữ liệu từ file Excel
df_before = pd.read_excel('total.xlsx', sheet_name="before")

# Định nghĩa biến phụ thuộc và các biến độc lập
X = df_before[['herding_behavior', 'loss_aversion', 'mental_accounting', 'overconfidence']]
y = df_before['abnormal_return']

# Thêm cột hằng số vào X để thực hiện hồi quy với intercept
X = sm.add_constant(X)

# Fit mô hình hồi quy tuyến tính
model = sm.OLS(y, X).fit()

confidence_intervals = model.conf_int(alpha=0.01)

# Hiển thị kết quả
print(model.summary())

# Trích xuất các thông số mô hình
model_summary = model.summary2().tables[1]  # Lấy bảng kết quả của mô hình

# Thêm các thông số khác vào DataFrame
additional_stats = {
    'R-squared': model.rsquared,
    'Adjusted R-squared': model.rsquared_adj,
    'Mean dependent var': y.mean,
    'S.D. dependent var': model.mse_resid**0.5,  # Standard deviation of residuals
    'S.E. of regression': model.bse.mean(),  # Standard error of regression
    'Sum squared resid': model.ssr,
    'Long-run variance': model.cov_HC0[0, 0]  # Estimate of long-run variance
}

# Chuyển bảng kết quả thành DataFrame
model_df_be = pd.DataFrame(model_summary)

# Thêm các thông số vào DataFrame
for stat, value in additional_stats.items():
    model_df_be.loc[stat] = value

# In bảng kết quả vào DataFrame
print(model_df_be)


                            OLS Regression Results                            
Dep. Variable:        abnormal_return   R-squared:                       0.769
Model:                            OLS   Adj. R-squared:                  0.768
Method:                 Least Squares   F-statistic:                     807.2
Date:                Mon, 30 Dec 2024   Prob (F-statistic):          4.46e-307
Time:                        11:05:09   Log-Likelihood:                -637.02
No. Observations:                 977   AIC:                             1284.
Df Residuals:                     972   BIC:                             1308.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                 0.0952      0.07

In [ ]:
import scipy
print(scipy.__version__)

In [ ]:
import statsmodels
print(statsmodels.__version__)

## during covid

In [17]:
# Đọc dữ liệu từ file Excel
df_during = pd.read_excel('total.xlsx', sheet_name="during")

# Định nghĩa biến phụ thuộc và các biến độc lập
X = df_during[['herding_behavior', 'loss_aversion', 'mental_accounting', 'overconfidence']]
y = df_during['abnormal_return']

# Thêm cột hằng số vào X để thực hiện hồi quy với intercept
X = sm.add_constant(X)

# Fit mô hình hồi quy tuyến tính
model = sm.OLS(y, X).fit()

# Hiển thị kết quả
print(model.summary())

# Trích xuất các thông số mô hình
model_summary = model.summary2().tables[1]  # Lấy bảng kết quả của mô hình

# Thêm các thông số khác vào DataFrame
additional_stats = {
    'R-squared': model.rsquared,
    'Adjusted R-squared': model.rsquared_adj,
    'Mean dependent var': y.mean,
    'S.D. dependent var': model.mse_resid**0.5,  # Standard deviation of residuals
    'S.E. of regression': model.bse.mean(),  # Standard error of regression
    'Sum squared resid': model.ssr,
    'Long-run variance': model.cov_HC0[0, 0]  # Estimate of long-run variance
}

# Chuyển bảng kết quả thành DataFrame
model_df_du = pd.DataFrame(model_summary)

# Thêm các thông số vào DataFrame
for stat, value in additional_stats.items():
    model_df_du.loc[stat] = value

# In bảng kết quả vào DataFrame
print(model_df_du)

                            OLS Regression Results                            
Dep. Variable:        abnormal_return   R-squared:                       0.679
Model:                            OLS   Adj. R-squared:                  0.676
Method:                 Least Squares   F-statistic:                     264.5
Date:                Mon, 30 Dec 2024   Prob (F-statistic):          5.29e-122
Time:                        10:54:59   Log-Likelihood:                -498.29
No. Observations:                 506   AIC:                             1007.
Df Residuals:                     501   BIC:                             1028.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.5234      0.30

## After covid

In [18]:
# Đọc dữ liệu từ file Excel
df_after = pd.read_excel('total.xlsx', sheet_name="after")

# Định nghĩa biến phụ thuộc và các biến độc lập
X = df_after[['herding_behavior', 'loss_aversion', 'mental_accounting', 'overconfidence']]
y = df_after['abnormal_return']

# Thêm cột hằng số vào X để thực hiện hồi quy với intercept
X = sm.add_constant(X)

# Fit mô hình hồi quy tuyến tính
model = sm.OLS(y, X).fit()


# Hiển thị kết quả
print(model.summary())

# Trích xuất các thông số mô hình
model_summary = model.summary2().tables[1]  # Lấy bảng kết quả của mô hình

# Thêm các thông số khác vào DataFrame
additional_stats = {
    'R-squared': model.rsquared,
    'Adjusted R-squared': model.rsquared_adj,
    'Mean dependent var': y.mean,
    'S.D. dependent var': model.mse_resid**0.5,  # Standard deviation of residuals
    'S.E. of regression': model.bse.mean(),  # Standard error of regression
    'Sum squared resid': model.ssr,
    'Long-run variance': model.cov_HC0[0, 0]  # Estimate of long-run variance
}

# Chuyển bảng kết quả thành DataFrame
model_df_af = pd.DataFrame(model_summary)

# Thêm các thông số vào DataFrame
for stat, value in additional_stats.items():
    model_df_af.loc[stat] = value

# In bảng kết quả vào DataFrame
print(model_df_af)

                            OLS Regression Results                            
Dep. Variable:        abnormal_return   R-squared:                       0.546
Model:                            OLS   Adj. R-squared:                  0.542
Method:                 Least Squares   F-statistic:                     140.3
Date:                Mon, 30 Dec 2024   Prob (F-statistic):           1.40e-78
Time:                        10:55:03   Log-Likelihood:                -377.21
No. Observations:                 471   AIC:                             764.4
Df Residuals:                     466   BIC:                             785.2
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -0.4126      0.32

## combine

In [19]:
combined_model_df = pd.concat([model_df_be,model_df_du,model_df_af], axis =0)

In [20]:
combined_model_df.to_excel(r'C:\Users\DELL\Desktop\financial materials\linear_regression_0.1.xlsx')

C:\Users\DELL\AppData\Local\Temp/ipykernel_20660/2127143256.py:1: UserWarning: Pandas requires version '3.0.5' or newer of 'xlsxwriter' (version '3.0.1' currently installed).
  combined_model_df.to_excel(r'C:\Users\DELL\Desktop\financial materials\linear_regression_0.1.xlsx')
